In [1]:
from keras.models import Sequential, load_model
import pandas as pd
import numpy as np
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import random
import datetime

from Indicator import Indicator

In [2]:
def fillindicator(df):
    indicator = Indicator()
    train_set = indicator.MACD(df,12,26,9,1)#MACD 1
    train_set = indicator.MACD(df,40,60,34,2)#MACD 2
    train_set = indicator.DMI(df,14)
    train_set = indicator.STOCHRSI(df)
    train_set = indicator.AO(df)
    train_set = indicator.WILLIANSR(df)
    train_set = indicator.RSI(df)
    train_set = indicator.MA(df,26)
    train_set['MA-26'] = train_set['MA']
    train_set = indicator.MA(df,15)
    train_set = indicator.check_trend(df)
    return train_set

def shiftdata(df,col_name,sft):
    for i in range(sft):
        i+=1
        shiftname = col_name+'-'+str(i)
        df[shiftname] = df[col_name].shift(i)

    return df

def norm(df,col_name):# this will get columnname + _n
    #Normalize data each colunm in to range -1 to 1
    df[col_name+'_n'] = 2*(df[col_name]-df[col_name].min())/(df[col_name].max()-df[col_name].min())-1    
    return df

def signal_to_catagorical(data):
    data.loc[data['Signal'] == 'wait ot hold', 'Signal'] = 'wait or hold'
    data.loc[data['Signal'] == 'buy', 'Signal'] = 0
    data.loc[data['Signal'] == 'sell', 'Signal'] = 1 
    data.loc[data['Signal'] == 'wait or hold', 'Signal'] = 2 
    return data

def add_shift_day(data,day):
    r_list = []
    for i in range(day,len(data)):#Number mean day to shift
        r_list.append(data[i-day : i])
    return np.array(r_list)

In [9]:
start = datetime.datetime(2018,10,1)#26 day before start year for make indicator
end = datetime.datetime(2020,1,1)

money = 100000
stocklist = ['AES','EBAY','VRTX','GOOGL','IP','CSCO','UAL','NVR','TPR','BA']

modellist = [f for f in glob.glob("Model/*.h5")]
print('have stock = ',len(modellist))
modellist

have stock =  6


['Model\\model_lstm3_1_19.h5',
 'Model\\model_lstm3_1_20.h5',
 'Model\\model_lstm_data20_2.h5',
 'Model\\model_lstm_data5_1000ep.h5',
 'Model\\model_lstm_data5_2000ep.h5',
 'Model\\model_lstm_data5_500ep.h5']

In [12]:
#Project model
modelstat = []
for md in modellist:
    PJmodel = []
    model = load_model(md)
    for stock in stocklist:
        df = web.DataReader(stock,'yahoo',start,end)
        df = fillindicator(df)
    #     df['AO-signal'] = np.where(df['AO'] > df['AO'].shift(1),1,0)
        df.loc[df['Trend'] == 'Down', 'Trend'] = 0.1
        df.loc[df['Trend'] == 'Up', 'Trend'] = 0.9 

        #Delta
        df['MACD-SL-1'] = df['MACD-1'] - df['SIGNAL LINE1']
        df['MACD-SL-2'] = df['MACD-2'] - df['SIGNAL LINE2']
        df['DMI'] = df['plusDI'] - df['minusDI']
        df['STOCHRSI'] = df['%K'] - df['%D']
        df['MA-trend'] = df['MA-26'] - df['Close']

        #MACD
        df = norm(df,'MACD-SL-1')
        df = norm(df,'MACD-SL-2')
        df = norm(df,'MACD-1')
        df = norm(df,'MACD-2')
        df = norm(df,'SIGNAL LINE1')
        df = norm(df,'SIGNAL LINE2')

        #DMI
        df = norm(df,'plusDI')
        df = norm(df,'minusDI')
        df = norm(df,'DMI')

        #StochasticRSI
        df = norm(df,'%K')
        df = norm(df,'%D')
        df = norm(df,'STOCHRSI')

        #Other
        df = norm(df,'Close')
        df = norm(df,'%R')
        df = norm(df,'RSI')
        df = norm(df,'AO')
        df = norm(df,'MA-trend')

        df = df[63:]
        p_test = df
        x_test = p_test[['MACD-SL-1_n','DMI_n','MACD-SL-2_n','Close_n','AO_n','RSI_n','MA-trend']].values
        x_test = add_shift_day(x_test,30)
        y_pred = model.predict(x_test)
        y_pred = np.argmax(y_pred,axis = -1)
        p_test = p_test.iloc[30:]

        p_test['pred'] = y_pred
        p_test['pred_b'] = np.where(p_test['pred'] == 0,p_test['Close'],np.nan)
        p_test['pred_s'] = np.where(p_test['pred'] == 1,p_test['Close'],np.nan)
        b = []
        s = []
        flag = 'sell'
        for i,r in p_test.iterrows():
            if (flag=='sell') & (r['pred']==0):
                b.append(r['Close'])
                flag = 'buy'
            elif (flag=='buy') & (r['pred']==1):
                s.append(r['Close'])
                flag = 'sell'
        if len(b)>len(s):
            del b[-1]
        modelbuy = len(b)
        modelsell = len(s)
        pfmodel = []
        profitmodel = []
        for i in range(len(b)):
            pfmodel.append(((s[i]-b[i])/(b[i]))*money)
            profitmodel.append(((s[i]-b[i])/(b[i])))
        if modelbuy >0:
            PJmodel.append([stock,modelbuy,sum(pfmodel)])    
        else:
            PJmodel.append([stock,0,0,0])    
    modelstat.append([md,PJmodel])

C:\Users\jame_\anaconda3\lib\site-packages\ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\jame_\anaconda3\lib\site-packages\ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\jame_\anaconda3\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [28]:
for i in range(len(modelstat[0])):
    print(modelstat[i])

['Model\\model_lstm3_1_19.h5', [['AES', 1, 12401.570984471002], ['EBAY', 0, 0, 0], ['VRTX', 0, 0, 0], ['GOOGL', 0, 0, 0], ['IP', 1, -1217.5483440375044], ['CSCO', 0, 0, 0], ['UAL', 0, 0, 0], ['NVR', 0, 0, 0], ['TPR', 1, 3197.280769572365], ['BA', 1, 7884.34966292042]]]
['Model\\model_lstm3_1_20.h5', [['AES', 0, 0, 0], ['EBAY', 0, 0, 0], ['VRTX', 1, 5914.426431206344], ['GOOGL', 1, 6679.975194280446], ['IP', 1, 6371.979724789273], ['CSCO', 1, 6197.559277480024], ['UAL', 2, 25090.476260265554], ['NVR', 0, 0, 0], ['TPR', 0, 0, 0], ['BA', 1, 4116.8803963432565]]]


In [29]:
modelstat

[['Model\\model_lstm3_1_19.h5',
  [['AES', 1, 12401.570984471002],
   ['EBAY', 0, 0, 0],
   ['VRTX', 0, 0, 0],
   ['GOOGL', 0, 0, 0],
   ['IP', 1, -1217.5483440375044],
   ['CSCO', 0, 0, 0],
   ['UAL', 0, 0, 0],
   ['NVR', 0, 0, 0],
   ['TPR', 1, 3197.280769572365],
   ['BA', 1, 7884.34966292042]]],
 ['Model\\model_lstm3_1_20.h5',
  [['AES', 0, 0, 0],
   ['EBAY', 0, 0, 0],
   ['VRTX', 1, 5914.426431206344],
   ['GOOGL', 1, 6679.975194280446],
   ['IP', 1, 6371.979724789273],
   ['CSCO', 1, 6197.559277480024],
   ['UAL', 2, 25090.476260265554],
   ['NVR', 0, 0, 0],
   ['TPR', 0, 0, 0],
   ['BA', 1, 4116.8803963432565]]],
 ['Model\\model_lstm_data20_2.h5',
  [['AES', 0, 0, 0],
   ['EBAY', 0, 0, 0],
   ['VRTX', 1, 32323.83685529851],
   ['GOOGL', 1, 8840.70304628276],
   ['IP', 1, -3713.8325148841855],
   ['CSCO', 0, 0, 0],
   ['UAL', 1, 13089.068881008665],
   ['NVR', 0, 0, 0],
   ['TPR', 0, 0, 0],
   ['BA', 1, 4724.742078614982]]],
 ['Model\\model_lstm_data5_1000ep.h5',
  [['AES', 5, -7